In [ ]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as sci_plt

from pprint import pprint
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv("data/stock_data.csv")
df.head()

,date,close,volume,open,high,low,symbol
0,2020-07-23,122.93,11010220,127.74,129.8494,121.32,SQ
1,2020-07-22,129.18,8437494,127.49,131.7100,127.06,SQ
2,2020-07-21,126.99,10787520,129.95,130.8000,125.07,SQ
3,2020-07-20,128.39,12423990,121.26,129.4600,120.53,SQ
4,2020-07-17,120.73,6968161,120.98,122.3800,118.59,SQ


In [5]:
# Get only the valid columns
df = df [['date', 'symbol', 'close']]

# Make symbol the headers (Pivot)
df = df.pivot(
    index='date',
    columns='symbol',
    values='close'
)

df.head()

symbol,AAPL,MSFT,SQ
date,,,
2020-01-24,318.31,165.04,71.14
2020-01-27,308.95,162.28,71.05
2020-01-28,317.69,165.46,74.80
2020-01-29,324.34,168.04,76.16
2020-01-30,323.87,172.78,76.10


In [13]:
# Print Functionality
def print_func(title: str, df: pd.DataFrame):
    print('')
    print('='*80)
    print(title)
    print('-'*80)
    print(df)
    print('-'*80)

In [15]:
# Calculate Log returns
log_return = np.log(1 + df.pct_change())

# Generate Random Weights (of each portfolio)
random_weights = np.array(np.random.random(3))
random_weights /= np.sum(random_weights) # Summing to 1

# Calculate Expected Return (Rp) for the whole year, i.e. 252 working days
rp = np.sum((log_return.mean() * random_weights) * 252)

# Calculate Expected Volatility, for whole year
exp_vol = np.sqrt(
    np.dot(
        random_weights.T,
        np.dot(
            log_return.cov() * 252,
            random_weights
        )
    )
)

# Calculate Sharpe Ratio
sharpe_ratio = rp / exp_vol

weights_df = pd.DataFrame(data={
    'random_weights': random_weights,
})
print_func('PORTFOLIO WEIGHTS:', weights_df)

metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': rp,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])
print_func('PORTFOLIO METRICS:', metrics_df)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
   random_weights
0        0.447689
1        0.498880
2        0.053431
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
--------------------------------------------------------------------------------
   Expected Portfolio Returns  Expected Portfolio Volatility  \
0                    0.404014                       0.543559   

   Portfolio Sharpe Ratio  
0                0.743277  
--------------------------------------------------------------------------------
